<a href="https://colab.research.google.com/github/esephaneli/fasion_stylist/blob/main/fasion_stylist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2 fotoğrafı modele “multimodal” gönderdiğimiz yer.

In [ ]:
content = [
  {"type": "input_text", "text": f"Kullanıcı tercihleri:\n{user_pref_text}\n\n{myntra_ctx}"},
  {"type": "input_image", "image_url": img_to_data_url(user_photo_path)},
  {"type": "input_image", "image_url": img_to_data_url(product_photo_path)},
  {"type": "input_text", "text": "Uyum + kombin önerisi ver."}
]


JSON Schema + strict ile “çıktıyı kilitlediğimiz" yer.

In [ ]:
schema = StyleAdvice.model_json_schema()
schema["required"] = list(schema.get("properties", {}).keys())
schema["additionalProperties"] = False

resp = client.responses.create(
  model="gpt-4o-mini",
  instructions=instructions,
  input=msgs,
  text={"format": {"type": "json_schema", "name": "style_advice", "schema": schema, "strict": True}},
)
out = StyleAdvice.model_validate_json(resp.output_text)


Pydantic ile output modeli

In [ ]:
class StyleAdvice(BaseModel):
    model_config = ConfigDict(extra="forbid")
    verdict: Literal["uygun","kismen_uygun","uygun_degil"]
    outfit_suggestions: List[str]
    confidence: float


Gradio arayüzü

In [ ]:
user_photo = gr.File(type="filepath", file_types=["image"])
product_photo = gr.File(type="filepath", file_types=["image"])
btn.click(run, inputs=[user_photo, product_photo, prefs, state], outputs=[result_text, result_json, state])


Kullanılan Dataset


```
https://www.kaggle.com/datasets/hiteshsuthar101/myntra-fashion-product-dataset
```

